In [81]:
import time
import os
import sys
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
import librosa
import soundfile as sf
import audioread
from pydub import AudioSegment
import resampy
import decorator
import six
import lazy_loader
from scipy.fftpack import dct




In [4]:
df=pd.read_csv("SleepAnalysis1.csv")

In [5]:
df

,Start Time,End Time,Sleep Efficiency (%),Total Sleep Time (Hours),Gender,Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),Exercise Duration (minutes),Insomnia (Yes/No)
0,12/29/2014 22:57,12/30/2014 7:30,1.00,8:33:00 AM,Male,59,5640,22.50,15,1,7,400,0,60,NO
1,12/30/2014 21:17,12/30/2014 21:33,0.03,12:16:00 AM,Male,72,4700,22.50,15,0,8,500,0,30,YES
2,12/30/2014 22:42,12/31/2014 7:13,0.98,8:30:00 AM,Male,57,3400,22.50,15,2,6,450,0,90,NO
3,12/31/2014 22:31,1/1/2015 6:03,0.65,7:32:00 AM,Male,60,8017,22.50,15,1,6,350,0,100,YES
4,1/1/2015 22:12,1/2/2015 4:56,0.72,6:44:00 AM,Male,68,2900,22.50,15,3,6,300,0,0,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,6/4/2018 22:05,6/5/2018 7:50,0.69,9:44:00 AM,Male,83,10873,21.85,10,4,3,227,4,0,YES
995,6/5/2018 23:27,6/6/2018 7:02,0.29,7:35:00 AM,Male,88,7284,21.85,10,3,3,876,1,0,YES
996,6/6/2018 23:13,6/7/2018 7:31,0.17,8:18:00 AM,Male,72,12669,21.85,10,1,5,553,1,0,YES
997,6/7/2018 22:13,6/8/2018 5:58,0.52,7:45:00 AM,Male,61,11763,21.85,10,3,4,440,0,0,YES


In [164]:
df_cat = df.select_dtypes(object)
df_num = df.select_dtypes(np.number)

In [165]:
df_num

,Sleep Efficiency (%),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),Exercise Duration (minutes)
0,1.00,59,5640,22.50,15,1,7,400,0,60
1,0.03,72,4700,22.50,15,0,8,500,0,30
2,0.98,57,3400,22.50,15,2,6,450,0,90
3,0.65,60,8017,22.50,15,1,6,350,0,100
4,0.72,68,2900,22.50,15,3,6,300,0,0
...,...,...,...,...,...,...,...,...,...,...
994,0.69,83,10873,21.85,10,4,3,227,4,0
995,0.29,88,7284,21.85,10,3,3,876,1,0
996,0.17,72,12669,21.85,10,1,5,553,1,0
997,0.52,61,11763,21.85,10,3,4,440,0,0


In [166]:
df_cat

,Start Time,End Time,Total Sleep Time (Hours),Gender,Insomnia (Yes/No)
0,12/29/2014 22:57,12/30/2014 7:30,8:33:00 AM,Male,NO
1,12/30/2014 21:17,12/30/2014 21:33,12:16:00 AM,Male,YES
2,12/30/2014 22:42,12/31/2014 7:13,8:30:00 AM,Male,NO
3,12/31/2014 22:31,1/1/2015 6:03,7:32:00 AM,Male,YES
4,1/1/2015 22:12,1/2/2015 4:56,6:44:00 AM,Male,YES
...,...,...,...,...,...
994,6/4/2018 22:05,6/5/2018 7:50,9:44:00 AM,Male,YES
995,6/5/2018 23:27,6/6/2018 7:02,7:35:00 AM,Male,YES
996,6/6/2018 23:13,6/7/2018 7:31,8:18:00 AM,Male,YES
997,6/7/2018 22:13,6/8/2018 5:58,7:45:00 AM,Male,YES


In [167]:
gender_dummies = pd.get_dummies(df_cat['Gender'], drop_first=True)

In [168]:
df_cat = pd.concat([df_cat, gender_dummies], axis=1)

In [169]:
df_cat

,Start Time,End Time,Total Sleep Time (Hours),Gender,Insomnia (Yes/No)
0,12/29/2014 22:57,12/30/2014 7:30,8:33:00 AM,Male,NO
1,12/30/2014 21:17,12/30/2014 21:33,12:16:00 AM,Male,YES
2,12/30/2014 22:42,12/31/2014 7:13,8:30:00 AM,Male,NO
3,12/31/2014 22:31,1/1/2015 6:03,7:32:00 AM,Male,YES
4,1/1/2015 22:12,1/2/2015 4:56,6:44:00 AM,Male,YES
...,...,...,...,...,...
994,6/4/2018 22:05,6/5/2018 7:50,9:44:00 AM,Male,YES
995,6/5/2018 23:27,6/6/2018 7:02,7:35:00 AM,Male,YES
996,6/6/2018 23:13,6/7/2018 7:31,8:18:00 AM,Male,YES
997,6/7/2018 22:13,6/8/2018 5:58,7:45:00 AM,Male,YES


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
df_one_hot = pd.get_dummies(df, columns = ['Gender', 'Insomnia (Yes/No)'])

In [8]:
df_one_hot

,Start Time,End Time,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),Exercise Duration (minutes),Gender_Male,Insomnia (Yes/No)_NO,Insomnia (Yes/No)_YES
0,12/29/2014 22:57,12/30/2014 7:30,1.00,8:33:00 AM,59,5640,22.50,15,1,7,400,0,60,True,True,False
1,12/30/2014 21:17,12/30/2014 21:33,0.03,12:16:00 AM,72,4700,22.50,15,0,8,500,0,30,True,False,True
2,12/30/2014 22:42,12/31/2014 7:13,0.98,8:30:00 AM,57,3400,22.50,15,2,6,450,0,90,True,True,False
3,12/31/2014 22:31,1/1/2015 6:03,0.65,7:32:00 AM,60,8017,22.50,15,1,6,350,0,100,True,False,True
4,1/1/2015 22:12,1/2/2015 4:56,0.72,6:44:00 AM,68,2900,22.50,15,3,6,300,0,0,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,6/4/2018 22:05,6/5/2018 7:50,0.69,9:44:00 AM,83,10873,21.85,10,4,3,227,4,0,True,False,True
995,6/5/2018 23:27,6/6/2018 7:02,0.29,7:35:00 AM,88,7284,21.85,10,3,3,876,1,0,True,False,True
996,6/6/2018 23:13,6/7/2018 7:31,0.17,8:18:00 AM,72,12669,21.85,10,1,5,553,1,0,True,False,True
997,6/7/2018 22:13,6/8/2018 5:58,0.52,7:45:00 AM,61,11763,21.85,10,3,4,440,0,0,True,False,True


In [9]:
for column in df_one_hot.columns:
    if df_one_hot[column].dtype == 'bool':
        df_one_hot[column] = df_one_hot[column].astype(int)

In [10]:
df_one_hot

,Start Time,End Time,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),Exercise Duration (minutes),Gender_Male,Insomnia (Yes/No)_NO,Insomnia (Yes/No)_YES
0,12/29/2014 22:57,12/30/2014 7:30,1.00,8:33:00 AM,59,5640,22.50,15,1,7,400,0,60,1,1,0
1,12/30/2014 21:17,12/30/2014 21:33,0.03,12:16:00 AM,72,4700,22.50,15,0,8,500,0,30,1,0,1
2,12/30/2014 22:42,12/31/2014 7:13,0.98,8:30:00 AM,57,3400,22.50,15,2,6,450,0,90,1,1,0
3,12/31/2014 22:31,1/1/2015 6:03,0.65,7:32:00 AM,60,8017,22.50,15,1,6,350,0,100,1,0,1
4,1/1/2015 22:12,1/2/2015 4:56,0.72,6:44:00 AM,68,2900,22.50,15,3,6,300,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,6/4/2018 22:05,6/5/2018 7:50,0.69,9:44:00 AM,83,10873,21.85,10,4,3,227,4,0,1,0,1
995,6/5/2018 23:27,6/6/2018 7:02,0.29,7:35:00 AM,88,7284,21.85,10,3,3,876,1,0,1,0,1
996,6/6/2018 23:13,6/7/2018 7:31,0.17,8:18:00 AM,72,12669,21.85,10,1,5,553,1,0,1,0,1
997,6/7/2018 22:13,6/8/2018 5:58,0.52,7:45:00 AM,61,11763,21.85,10,3,4,440,0,0,1,0,1


In [11]:
df_one_hot['Start Time'] = pd.to_datetime(df_one_hot['Start Time'])
df_one_hot['End Time'] = pd.to_datetime(df_one_hot['End Time'])

In [12]:
df_one_hot['Start Year'] = df_one_hot['Start Time'].dt.year
df_one_hot['Start Month'] = df_one_hot['Start Time'].dt.month
df_one_hot['Start Day'] = df_one_hot['Start Time'].dt.day
df_one_hot['Start Hour'] = df_one_hot['Start Time'].dt.hour
df_one_hot['Start Minute'] = df_one_hot['Start Time'].dt.minute
df_one_hot['Start Second'] = df_one_hot['Start Time'].dt.second

In [13]:
df_one_hot['End Year'] = df_one_hot['End Time'].dt.year
df_one_hot['End Month'] = df_one_hot['End Time'].dt.month
df_one_hot['End Day'] = df_one_hot['End Time'].dt.day
df_one_hot['End Hour'] = df_one_hot['End Time'].dt.hour
df_one_hot['End Minute'] = df_one_hot['End Time'].dt.minute
df_one_hot['End Second'] = df_one_hot['End Time'].dt.second

In [14]:
df_one_hot['Start Day Of Week'] = df_one_hot['Start Time'].dt.dayofweek
df_one_hot['End Day Of Week'] = df_one_hot['End Time'].dt.dayofweek

In [15]:
df_one_hot

,Start Time,End Time,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),...,Start Minute,Start Second,End Year,End Month,End Day,End Hour,End Minute,End Second,Start Day Of Week,End Day Of Week
0,2014-12-29 22:57:00,2014-12-30 07:30:00,1.00,8:33:00 AM,59,5640,22.50,15,1,7,...,57,0,2014,12,30,7,30,0,0,1
1,2014-12-30 21:17:00,2014-12-30 21:33:00,0.03,12:16:00 AM,72,4700,22.50,15,0,8,...,17,0,2014,12,30,21,33,0,1,1
2,2014-12-30 22:42:00,2014-12-31 07:13:00,0.98,8:30:00 AM,57,3400,22.50,15,2,6,...,42,0,2014,12,31,7,13,0,1,2
3,2014-12-31 22:31:00,2015-01-01 06:03:00,0.65,7:32:00 AM,60,8017,22.50,15,1,6,...,31,0,2015,1,1,6,3,0,2,3
4,2015-01-01 22:12:00,2015-01-02 04:56:00,0.72,6:44:00 AM,68,2900,22.50,15,3,6,...,12,0,2015,1,2,4,56,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,2018-06-04 22:05:00,2018-06-05 07:50:00,0.69,9:44:00 AM,83,10873,21.85,10,4,3,...,5,0,2018,6,5,7,50,0,0,1
995,2018-06-05 23:27:00,2018-06-06 07:02:00,0.29,7:35:00 AM,88,7284,21.85,10,3,3,...,27,0,2018,6,6,7,2,0,1,2
996,2018-06-06 23:13:00,2018-06-07 07:31:00,0.17,8:18:00 AM,72,12669,21.85,10,1,5,...,13,0,2018,6,7,7,31,0,2,3
997,2018-06-07 22:13:00,2018-06-08 05:58:00,0.52,7:45:00 AM,61,11763,21.85,10,3,4,...,13,0,2018,6,8,5,58,0,3,4


In [16]:
def parse_time(x):
    formats = ['%I:%M:%S %p', '%I:%M %p', '%I %p']
    for fmt in formats:
        try:
            return pd.to_datetime(x, format=fmt)
        except ValueError:
            continue
    try:
        # If the value is not in any expected format, try to parse it as a float (e.g., '8.55')
        return pd.to_datetime(float(x), unit='h')
    except ValueError:
        raise ValueError(f"Time data '{x}' does not match any known format.")

In [17]:
df_one_hot['Total Sleep Time (Hours)'] = df_one_hot['Total Sleep Time (Hours)'].apply(parse_time)

In [18]:
df_one_hot['Total Sleep Time (Hours)'] = df_one_hot['Total Sleep Time (Hours)'].dt.hour + df_one_hot['Total Sleep Time (Hours)'].dt.minute / 60 + df_one_hot['Total Sleep Time (Hours)'].dt.second / 3600

In [19]:
df_one_hot

,Start Time,End Time,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),...,Start Minute,Start Second,End Year,End Month,End Day,End Hour,End Minute,End Second,Start Day Of Week,End Day Of Week
0,2014-12-29 22:57:00,2014-12-30 07:30:00,1.00,8.550000,59,5640,22.50,15,1,7,...,57,0,2014,12,30,7,30,0,0,1
1,2014-12-30 21:17:00,2014-12-30 21:33:00,0.03,0.266667,72,4700,22.50,15,0,8,...,17,0,2014,12,30,21,33,0,1,1
2,2014-12-30 22:42:00,2014-12-31 07:13:00,0.98,8.500000,57,3400,22.50,15,2,6,...,42,0,2014,12,31,7,13,0,1,2
3,2014-12-31 22:31:00,2015-01-01 06:03:00,0.65,7.533333,60,8017,22.50,15,1,6,...,31,0,2015,1,1,6,3,0,2,3
4,2015-01-01 22:12:00,2015-01-02 04:56:00,0.72,6.733333,68,2900,22.50,15,3,6,...,12,0,2015,1,2,4,56,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,2018-06-04 22:05:00,2018-06-05 07:50:00,0.69,9.733333,83,10873,21.85,10,4,3,...,5,0,2018,6,5,7,50,0,0,1
995,2018-06-05 23:27:00,2018-06-06 07:02:00,0.29,7.583333,88,7284,21.85,10,3,3,...,27,0,2018,6,6,7,2,0,1,2
996,2018-06-06 23:13:00,2018-06-07 07:31:00,0.17,8.300000,72,12669,21.85,10,1,5,...,13,0,2018,6,7,7,31,0,2,3
997,2018-06-07 22:13:00,2018-06-08 05:58:00,0.52,7.750000,61,11763,21.85,10,3,4,...,13,0,2018,6,8,5,58,0,3,4


In [20]:
df_one_hot.drop(columns=['Start Time', 'End Time'], inplace=True)

In [21]:
df_one_hot

,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),...,Start Minute,Start Second,End Year,End Month,End Day,End Hour,End Minute,End Second,Start Day Of Week,End Day Of Week
0,1.00,8.550000,59,5640,22.50,15,1,7,400,0,...,57,0,2014,12,30,7,30,0,0,1
1,0.03,0.266667,72,4700,22.50,15,0,8,500,0,...,17,0,2014,12,30,21,33,0,1,1
2,0.98,8.500000,57,3400,22.50,15,2,6,450,0,...,42,0,2014,12,31,7,13,0,1,2
3,0.65,7.533333,60,8017,22.50,15,1,6,350,0,...,31,0,2015,1,1,6,3,0,2,3
4,0.72,6.733333,68,2900,22.50,15,3,6,300,0,...,12,0,2015,1,2,4,56,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0.69,9.733333,83,10873,21.85,10,4,3,227,4,...,5,0,2018,6,5,7,50,0,0,1
995,0.29,7.583333,88,7284,21.85,10,3,3,876,1,...,27,0,2018,6,6,7,2,0,1,2
996,0.17,8.300000,72,12669,21.85,10,1,5,553,1,...,13,0,2018,6,7,7,31,0,2,3
997,0.52,7.750000,61,11763,21.85,10,3,4,440,0,...,13,0,2018,6,8,5,58,0,3,4


In [22]:
from sklearn.metrics import accuracy_score

In [23]:
from sklearn.model_selection import cross_val_score

In [24]:
from sklearn.metrics import precision_score

In [25]:
from sklearn.metrics import recall_score

In [26]:
from sklearn.metrics import classification_report

In [28]:
from sklearn.metrics import roc_curve, auc

In [29]:
import os

In [30]:
df_one_hot

,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),...,Start Minute,Start Second,End Year,End Month,End Day,End Hour,End Minute,End Second,Start Day Of Week,End Day Of Week
0,1.00,8.550000,59,5640,22.50,15,1,7,400,0,...,57,0,2014,12,30,7,30,0,0,1
1,0.03,0.266667,72,4700,22.50,15,0,8,500,0,...,17,0,2014,12,30,21,33,0,1,1
2,0.98,8.500000,57,3400,22.50,15,2,6,450,0,...,42,0,2014,12,31,7,13,0,1,2
3,0.65,7.533333,60,8017,22.50,15,1,6,350,0,...,31,0,2015,1,1,6,3,0,2,3
4,0.72,6.733333,68,2900,22.50,15,3,6,300,0,...,12,0,2015,1,2,4,56,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0.69,9.733333,83,10873,21.85,10,4,3,227,4,...,5,0,2018,6,5,7,50,0,0,1
995,0.29,7.583333,88,7284,21.85,10,3,3,876,1,...,27,0,2018,6,6,7,2,0,1,2
996,0.17,8.300000,72,12669,21.85,10,1,5,553,1,...,13,0,2018,6,7,7,31,0,2,3
997,0.52,7.750000,61,11763,21.85,10,3,4,440,0,...,13,0,2018,6,8,5,58,0,3,4


In [31]:
df_one_hot

,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),...,Start Minute,Start Second,End Year,End Month,End Day,End Hour,End Minute,End Second,Start Day Of Week,End Day Of Week
0,1.00,8.550000,59,5640,22.50,15,1,7,400,0,...,57,0,2014,12,30,7,30,0,0,1
1,0.03,0.266667,72,4700,22.50,15,0,8,500,0,...,17,0,2014,12,30,21,33,0,1,1
2,0.98,8.500000,57,3400,22.50,15,2,6,450,0,...,42,0,2014,12,31,7,13,0,1,2
3,0.65,7.533333,60,8017,22.50,15,1,6,350,0,...,31,0,2015,1,1,6,3,0,2,3
4,0.72,6.733333,68,2900,22.50,15,3,6,300,0,...,12,0,2015,1,2,4,56,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0.69,9.733333,83,10873,21.85,10,4,3,227,4,...,5,0,2018,6,5,7,50,0,0,1
995,0.29,7.583333,88,7284,21.85,10,3,3,876,1,...,27,0,2018,6,6,7,2,0,1,2
996,0.17,8.300000,72,12669,21.85,10,1,5,553,1,...,13,0,2018,6,7,7,31,0,2,3
997,0.52,7.750000,61,11763,21.85,10,3,4,440,0,...,13,0,2018,6,8,5,58,0,3,4


In [32]:
folder_path = r'C:\Users\HP\1'

In [33]:
files = os.listdir(folder_path)

In [34]:
files

['1_0.wav',
 '1_1.wav',
 '1_10.wav',
 '1_100.wav',
 '1_101.wav',
 '1_102.wav',
 '1_103.wav',
 '1_104.wav',
 '1_105.wav',
 '1_106.wav',
 '1_107.wav',
 '1_108.wav',
 '1_109.wav',
 '1_11.wav',
 '1_110.wav',
 '1_111.wav',
 '1_112.wav',
 '1_113.wav',
 '1_114.wav',
 '1_115.wav',
 '1_116.wav',
 '1_117.wav',
 '1_118.wav',
 '1_119.wav',
 '1_12.wav',
 '1_120.wav',
 '1_121.wav',
 '1_122.wav',
 '1_123.wav',
 '1_124.wav',
 '1_125.wav',
 '1_126.wav',
 '1_127.wav',
 '1_128.wav',
 '1_129.wav',
 '1_13.wav',
 '1_130.wav',
 '1_131.wav',
 '1_132.wav',
 '1_133.wav',
 '1_134.wav',
 '1_135.wav',
 '1_136.wav',
 '1_137.wav',
 '1_138.wav',
 '1_139.wav',
 '1_14.wav',
 '1_140.wav',
 '1_141.wav',
 '1_142.wav',
 '1_143.wav',
 '1_144.wav',
 '1_145.wav',
 '1_146.wav',
 '1_147.wav',
 '1_148.wav',
 '1_149.wav',
 '1_15.wav',
 '1_150.wav',
 '1_151.wav',
 '1_152.wav',
 '1_153.wav',
 '1_154.wav',
 '1_155.wav',
 '1_156.wav',
 '1_157.wav',
 '1_158.wav',
 '1_159.wav',
 '1_16.wav',
 '1_160.wav',
 '1_161.wav',
 '1_162.wav',
 '1

In [35]:
wav_files = [file for file in files if file.endswith('.wav')]

In [45]:
df_one_hot['snoring_sound_path'] = [os.path.join(folder_path, file) for file in wav_files]

In [46]:
df_one_hot = df_one_hot.iloc[:len(wav_files), :]

In [47]:
df_one_hot

,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),...,End Year,End Month,End Day,End Hour,End Minute,End Second,Start Day Of Week,End Day Of Week,non-snoring_sound_path,snoring_sound_path
0,1.00,8.550000,59,5640,22.5,15,1,7,400,0,...,2014,12,30,7,30,0,0,1,C:\Users\HP\0\0_0.wav,C:\Users\HP\1\1_0.wav
1,0.03,0.266667,72,4700,22.5,15,0,8,500,0,...,2014,12,30,21,33,0,1,1,C:\Users\HP\0\0_1.wav,C:\Users\HP\1\1_1.wav
2,0.98,8.500000,57,3400,22.5,15,2,6,450,0,...,2014,12,31,7,13,0,1,2,C:\Users\HP\0\0_10.wav,C:\Users\HP\1\1_10.wav
3,0.65,7.533333,60,8017,22.5,15,1,6,350,0,...,2015,1,1,6,3,0,2,3,C:\Users\HP\0\0_100.wav,C:\Users\HP\1\1_100.wav
4,0.72,6.733333,68,2900,22.5,15,3,6,300,0,...,2015,1,2,4,56,0,3,4,C:\Users\HP\0\0_101.wav,C:\Users\HP\1\1_101.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.88,8.650000,74,6536,21.0,20,1,6,400,4,...,2016,11,25,6,39,0,3,4,C:\Users\HP\0\0_95.wav,C:\Users\HP\1\1_95.wav
496,0.89,8.500000,74,5628,21.0,20,1,5,300,4,...,2016,11,26,7,40,0,4,5,C:\Users\HP\0\0_96.wav,C:\Users\HP\1\1_96.wav
497,0.75,8.266667,74,595,21.0,20,1,5,300,2,...,2016,11,28,6,18,0,6,0,C:\Users\HP\0\0_97.wav,C:\Users\HP\1\1_97.wav
498,0.83,7.833333,74,7499,21.0,20,1,4,200,2,...,2016,11,29,6,26,0,0,1,C:\Users\HP\0\0_98.wav,C:\Users\HP\1\1_98.wav


In [38]:
folder_path_non_snoring = r'C:\Users\HP\0'

In [39]:
files_non_snoring = os.listdir(folder_path_non_snoring)

In [40]:
files

['1_0.wav',
 '1_1.wav',
 '1_10.wav',
 '1_100.wav',
 '1_101.wav',
 '1_102.wav',
 '1_103.wav',
 '1_104.wav',
 '1_105.wav',
 '1_106.wav',
 '1_107.wav',
 '1_108.wav',
 '1_109.wav',
 '1_11.wav',
 '1_110.wav',
 '1_111.wav',
 '1_112.wav',
 '1_113.wav',
 '1_114.wav',
 '1_115.wav',
 '1_116.wav',
 '1_117.wav',
 '1_118.wav',
 '1_119.wav',
 '1_12.wav',
 '1_120.wav',
 '1_121.wav',
 '1_122.wav',
 '1_123.wav',
 '1_124.wav',
 '1_125.wav',
 '1_126.wav',
 '1_127.wav',
 '1_128.wav',
 '1_129.wav',
 '1_13.wav',
 '1_130.wav',
 '1_131.wav',
 '1_132.wav',
 '1_133.wav',
 '1_134.wav',
 '1_135.wav',
 '1_136.wav',
 '1_137.wav',
 '1_138.wav',
 '1_139.wav',
 '1_14.wav',
 '1_140.wav',
 '1_141.wav',
 '1_142.wav',
 '1_143.wav',
 '1_144.wav',
 '1_145.wav',
 '1_146.wav',
 '1_147.wav',
 '1_148.wav',
 '1_149.wav',
 '1_15.wav',
 '1_150.wav',
 '1_151.wav',
 '1_152.wav',
 '1_153.wav',
 '1_154.wav',
 '1_155.wav',
 '1_156.wav',
 '1_157.wav',
 '1_158.wav',
 '1_159.wav',
 '1_16.wav',
 '1_160.wav',
 '1_161.wav',
 '1_162.wav',
 '1

In [41]:
wav_files_non_snoring = [file for file in files_non_snoring if file.endswith('.wav')]

In [42]:
df_one_hot['non-snoring_sound_path'] = [os.path.join(folder_path_non_snoring, file) for file in wav_files_non_snoring]

In [43]:
df_one_hot = df_one_hot.iloc[:len(wav_files_non_snoring), :]

In [44]:
df_one_hot

,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),...,Start Second,End Year,End Month,End Day,End Hour,End Minute,End Second,Start Day Of Week,End Day Of Week,non-snoring_sound_path
0,1.00,8.550000,59,5640,22.5,15,1,7,400,0,...,0,2014,12,30,7,30,0,0,1,C:\Users\HP\0\0_0.wav
1,0.03,0.266667,72,4700,22.5,15,0,8,500,0,...,0,2014,12,30,21,33,0,1,1,C:\Users\HP\0\0_1.wav
2,0.98,8.500000,57,3400,22.5,15,2,6,450,0,...,0,2014,12,31,7,13,0,1,2,C:\Users\HP\0\0_10.wav
3,0.65,7.533333,60,8017,22.5,15,1,6,350,0,...,0,2015,1,1,6,3,0,2,3,C:\Users\HP\0\0_100.wav
4,0.72,6.733333,68,2900,22.5,15,3,6,300,0,...,0,2015,1,2,4,56,0,3,4,C:\Users\HP\0\0_101.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.88,8.650000,74,6536,21.0,20,1,6,400,4,...,0,2016,11,25,6,39,0,3,4,C:\Users\HP\0\0_95.wav
496,0.89,8.500000,74,5628,21.0,20,1,5,300,4,...,0,2016,11,26,7,40,0,4,5,C:\Users\HP\0\0_96.wav
497,0.75,8.266667,74,595,21.0,20,1,5,300,2,...,0,2016,11,28,6,18,0,6,0,C:\Users\HP\0\0_97.wav
498,0.83,7.833333,74,7499,21.0,20,1,4,200,2,...,0,2016,11,29,6,26,0,0,1,C:\Users\HP\0\0_98.wav


In [217]:
df_one_hot

,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),...,End Year,End Month,End Day,End Hour,End Minute,End Second,Start Day Of Week,End Day Of Week,snoring_sound_path,non-snoring_sound_path
0,1.00,8.550000,59,5640,22.5,15,1,7,400,0,...,2014,12,30,7,30,0,0,1,C:\Users\HP\1\1_0.wav,C:\Users\HP\0\0_0.wav
1,0.03,0.266667,72,4700,22.5,15,0,8,500,0,...,2014,12,30,21,33,0,1,1,C:\Users\HP\1\1_1.wav,C:\Users\HP\0\0_1.wav
2,0.98,8.500000,57,3400,22.5,15,2,6,450,0,...,2014,12,31,7,13,0,1,2,C:\Users\HP\1\1_10.wav,C:\Users\HP\0\0_10.wav
3,0.65,7.533333,60,8017,22.5,15,1,6,350,0,...,2015,1,1,6,3,0,2,3,C:\Users\HP\1\1_100.wav,C:\Users\HP\0\0_100.wav
4,0.72,6.733333,68,2900,22.5,15,3,6,300,0,...,2015,1,2,4,56,0,3,4,C:\Users\HP\1\1_101.wav,C:\Users\HP\0\0_101.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.88,8.650000,74,6536,21.0,20,1,6,400,4,...,2016,11,25,6,39,0,3,4,C:\Users\HP\1\1_95.wav,C:\Users\HP\0\0_95.wav
496,0.89,8.500000,74,5628,21.0,20,1,5,300,4,...,2016,11,26,7,40,0,4,5,C:\Users\HP\1\1_96.wav,C:\Users\HP\0\0_96.wav
497,0.75,8.266667,74,595,21.0,20,1,5,300,2,...,2016,11,28,6,18,0,6,0,C:\Users\HP\1\1_97.wav,C:\Users\HP\0\0_97.wav
498,0.83,7.833333,74,7499,21.0,20,1,4,200,2,...,2016,11,29,6,26,0,0,1,C:\Users\HP\1\1_98.wav,C:\Users\HP\0\0_98.wav


In [49]:
def extract_mfcc(signal, samplerate=16000, numcep=13, nfft=512):
    # Pre-emphasis
    pre_emphasis = 0.97
    emphasized_signal = np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])

    # Framing
    frame_size = 0.025
    frame_stride = 0.01
    frame_length, frame_step = frame_size * samplerate, frame_stride * samplerate
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))

    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(emphasized_signal, z)

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]

    # Window
    frames *= np.hamming(frame_length)

    # Fourier-Transform and Power Spectrum
    mag_frames = np.absolute(np.fft.rfft(frames, nfft))
    pow_frames = ((1.0 / nfft) * ((mag_frames) ** 2))

    # Filter Banks
    nfilt = 40
    low_freq_mel = 0
    high_freq_mel = (2595 * np.log10(1 + (samplerate / 2) / 700))
    mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)
    hz_points = (700 * (10**(mel_points / 2595) - 1))
    bin = np.floor((nfft + 1) * hz_points / samplerate)

    fbank = np.zeros((nfilt, int(np.floor(nfft / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])
        f_m = int(bin[m])
        f_m_plus = int(bin[m + 1])

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])

    filter_banks = np.dot(pow_frames, fbank.T)
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)
    filter_banks = 20 * np.log10(filter_banks)

    # MFCC
    mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, :numcep]
    return np.mean(mfcc, axis=0)

def extract_features(file_path):
    try:
        # Load the audio file using pydub
        audio = AudioSegment.from_wav(file_path)
        # Convert to raw audio data
        audio_data = np.array(audio.get_array_of_samples(), dtype=np.float32)
        sr = audio.frame_rate

        # Normalize audio data
        if audio.channels == 2:
            audio_data = audio_data.reshape((-1, 2)).mean(axis=1)

        # Extract MFCC features
        mfccs_mean = extract_mfcc(audio_data, samplerate=sr)

    except Exception as e:
        print(f"Error extracting features from {file_path}: {e}")
        return None
    return mfccs_mean

In [50]:
df_one_hot['snoring_mfcc'] = df_one_hot['snoring_sound_path'].apply(extract_features)

In [51]:
df_one_hot['non-snoring_mfcc'] = df_one_hot['non-snoring_sound_path'].apply(extract_features)

In [52]:
df_one_hot

,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),...,End Day,End Hour,End Minute,End Second,Start Day Of Week,End Day Of Week,non-snoring_sound_path,snoring_sound_path,snoring_mfcc,non-snoring_mfcc
0,1.00,8.550000,59,5640,22.5,15,1,7,400,0,...,30,7,30,0,0,1,C:\Users\HP\0\0_0.wav,C:\Users\HP\1\1_0.wav,"[451.31067379115797, -12.855848265242065, -78....","[573.6100691133169, -25.259349499149646, -75.2..."
1,0.03,0.266667,72,4700,22.5,15,0,8,500,0,...,30,21,33,0,1,1,C:\Users\HP\0\0_1.wav,C:\Users\HP\1\1_1.wav,"[312.3022413893721, 3.7233212263186775, -59.27...","[679.4183866304622, -53.47156914491609, -113.3..."
2,0.98,8.500000,57,3400,22.5,15,2,6,450,0,...,31,7,13,0,1,2,C:\Users\HP\0\0_10.wav,C:\Users\HP\1\1_10.wav,"[428.5039259960272, -4.58086182961318, -45.462...","[628.7005352993434, -36.486629878975485, -107...."
3,0.65,7.533333,60,8017,22.5,15,1,6,350,0,...,1,6,3,0,2,3,C:\Users\HP\0\0_100.wav,C:\Users\HP\1\1_100.wav,"[294.1051127380376, 24.83682502237962, -40.771...","[-182.29836761492265, 73.1860879297754, 29.430..."
4,0.72,6.733333,68,2900,22.5,15,3,6,300,0,...,2,4,56,0,3,4,C:\Users\HP\0\0_101.wav,C:\Users\HP\1\1_101.wav,"[454.5533007861852, 67.41967675586928, -84.586...","[300.9906034626255, 103.87181548063303, 35.805..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.88,8.650000,74,6536,21.0,20,1,6,400,4,...,25,6,39,0,3,4,C:\Users\HP\0\0_95.wav,C:\Users\HP\1\1_95.wav,"[444.4476651438037, 68.09231245055149, -75.481...","[572.2055934406125, -88.33046116882568, 20.564..."
496,0.89,8.500000,74,5628,21.0,20,1,5,300,4,...,26,7,40,0,4,5,C:\Users\HP\0\0_96.wav,C:\Users\HP\1\1_96.wav,"[451.58656222139064, 65.94332218546501, -83.10...","[590.3827310780678, -79.38787522664433, 20.189..."
497,0.75,8.266667,74,595,21.0,20,1,5,300,2,...,28,6,18,0,6,0,C:\Users\HP\0\0_97.wav,C:\Users\HP\1\1_97.wav,"[223.64504452025724, 10.566871177700259, -44.3...","[557.2476437847927, -81.79487138727038, 24.233..."
498,0.83,7.833333,74,7499,21.0,20,1,4,200,2,...,29,6,26,0,0,1,C:\Users\HP\0\0_98.wav,C:\Users\HP\1\1_98.wav,"[452.87667146465776, 69.84529571976965, -80.01...","[593.0008949219119, -65.70009829564775, 16.641..."


In [53]:
n_mfcc = 13

snoring_mfcc_df = pd.DataFrame(df_one_hot['snoring_mfcc'].tolist(), columns=[f'snoring_mfcc_{i}' for i in range(n_mfcc)])
non_snoring_mfcc_df = pd.DataFrame(df_one_hot['non-snoring_mfcc'].tolist(), columns=[f'non_snoring_mfcc_{i}' for i in range(n_mfcc)])

In [54]:
df_one_hot

,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),...,End Day,End Hour,End Minute,End Second,Start Day Of Week,End Day Of Week,non-snoring_sound_path,snoring_sound_path,snoring_mfcc,non-snoring_mfcc
0,1.00,8.550000,59,5640,22.5,15,1,7,400,0,...,30,7,30,0,0,1,C:\Users\HP\0\0_0.wav,C:\Users\HP\1\1_0.wav,"[451.31067379115797, -12.855848265242065, -78....","[573.6100691133169, -25.259349499149646, -75.2..."
1,0.03,0.266667,72,4700,22.5,15,0,8,500,0,...,30,21,33,0,1,1,C:\Users\HP\0\0_1.wav,C:\Users\HP\1\1_1.wav,"[312.3022413893721, 3.7233212263186775, -59.27...","[679.4183866304622, -53.47156914491609, -113.3..."
2,0.98,8.500000,57,3400,22.5,15,2,6,450,0,...,31,7,13,0,1,2,C:\Users\HP\0\0_10.wav,C:\Users\HP\1\1_10.wav,"[428.5039259960272, -4.58086182961318, -45.462...","[628.7005352993434, -36.486629878975485, -107...."
3,0.65,7.533333,60,8017,22.5,15,1,6,350,0,...,1,6,3,0,2,3,C:\Users\HP\0\0_100.wav,C:\Users\HP\1\1_100.wav,"[294.1051127380376, 24.83682502237962, -40.771...","[-182.29836761492265, 73.1860879297754, 29.430..."
4,0.72,6.733333,68,2900,22.5,15,3,6,300,0,...,2,4,56,0,3,4,C:\Users\HP\0\0_101.wav,C:\Users\HP\1\1_101.wav,"[454.5533007861852, 67.41967675586928, -84.586...","[300.9906034626255, 103.87181548063303, 35.805..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.88,8.650000,74,6536,21.0,20,1,6,400,4,...,25,6,39,0,3,4,C:\Users\HP\0\0_95.wav,C:\Users\HP\1\1_95.wav,"[444.4476651438037, 68.09231245055149, -75.481...","[572.2055934406125, -88.33046116882568, 20.564..."
496,0.89,8.500000,74,5628,21.0,20,1,5,300,4,...,26,7,40,0,4,5,C:\Users\HP\0\0_96.wav,C:\Users\HP\1\1_96.wav,"[451.58656222139064, 65.94332218546501, -83.10...","[590.3827310780678, -79.38787522664433, 20.189..."
497,0.75,8.266667,74,595,21.0,20,1,5,300,2,...,28,6,18,0,6,0,C:\Users\HP\0\0_97.wav,C:\Users\HP\1\1_97.wav,"[223.64504452025724, 10.566871177700259, -44.3...","[557.2476437847927, -81.79487138727038, 24.233..."
498,0.83,7.833333,74,7499,21.0,20,1,4,200,2,...,29,6,26,0,0,1,C:\Users\HP\0\0_98.wav,C:\Users\HP\1\1_98.wav,"[452.87667146465776, 69.84529571976965, -80.01...","[593.0008949219119, -65.70009829564775, 16.641..."


In [55]:
df_one_hot = pd.concat([df_one_hot, snoring_mfcc_df, non_snoring_mfcc_df], axis=1)

In [56]:
df_one_hot

,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),...,non_snoring_mfcc_3,non_snoring_mfcc_4,non_snoring_mfcc_5,non_snoring_mfcc_6,non_snoring_mfcc_7,non_snoring_mfcc_8,non_snoring_mfcc_9,non_snoring_mfcc_10,non_snoring_mfcc_11,non_snoring_mfcc_12
0,1.00,8.550000,59,5640,22.5,15,1,7,400,0,...,-9.269381,-18.835488,20.378508,-11.392241,20.031550,-2.818156,-2.804502,6.367196,-2.627417,0.926880
1,0.03,0.266667,72,4700,22.5,15,0,8,500,0,...,13.897652,-1.592268,23.600391,-23.501267,6.519634,5.660507,-2.793761,12.978388,-9.589830,-7.780554
2,0.98,8.500000,57,3400,22.5,15,2,6,450,0,...,-4.694515,-2.959561,21.859583,-36.483370,19.489034,1.779462,-4.483938,-4.372021,5.027823,-6.741121
3,0.65,7.533333,60,8017,22.5,15,1,6,350,0,...,22.115048,6.243104,10.195583,3.265523,4.448919,4.378773,2.345736,1.257296,-3.991786,-3.043323
4,0.72,6.733333,68,2900,22.5,15,3,6,300,0,...,26.607399,7.497952,12.391656,5.850002,4.529510,6.236501,4.371455,1.426839,-2.397728,-4.159834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.88,8.650000,74,6536,21.0,20,1,6,400,4,...,55.781423,-34.890406,24.910494,-24.711970,31.871810,-24.889958,22.753106,-11.470411,7.938448,-7.446821
496,0.89,8.500000,74,5628,21.0,20,1,5,300,4,...,47.145010,-41.015930,19.350729,-28.172226,30.483321,-25.031947,27.416434,-7.228113,12.078870,-8.351442
497,0.75,8.266667,74,595,21.0,20,1,5,300,2,...,53.863180,-32.002747,27.553545,-24.016376,35.167236,-25.989636,23.615290,-13.277489,11.329945,-10.064979
498,0.83,7.833333,74,7499,21.0,20,1,4,200,2,...,36.533682,-44.521168,20.733773,-28.723805,31.096191,-21.767701,26.089461,-10.327917,11.956299,-10.462354


In [61]:
df_one_hot

,Sleep Efficiency (%),Total Sleep Time (Hours),Heart Rate (BPM),Activity Steps,Body Mass Index (BMI),Sleep Onset Latency (Minutes),Number of wake episodes,Stress Level (scale 1 - 10),Caffeine Intake (mg),Alcohol Consumption (Units),...,non_snoring_mfcc_3,non_snoring_mfcc_4,non_snoring_mfcc_5,non_snoring_mfcc_6,non_snoring_mfcc_7,non_snoring_mfcc_8,non_snoring_mfcc_9,non_snoring_mfcc_10,non_snoring_mfcc_11,non_snoring_mfcc_12
0,1.00,8.550000,59,5640,22.5,15,1,7,400,0,...,-9.269381,-18.835488,20.378508,-11.392241,20.031550,-2.818156,-2.804502,6.367196,-2.627417,0.926880
1,0.03,0.266667,72,4700,22.5,15,0,8,500,0,...,13.897652,-1.592268,23.600391,-23.501267,6.519634,5.660507,-2.793761,12.978388,-9.589830,-7.780554
2,0.98,8.500000,57,3400,22.5,15,2,6,450,0,...,-4.694515,-2.959561,21.859583,-36.483370,19.489034,1.779462,-4.483938,-4.372021,5.027823,-6.741121
3,0.65,7.533333,60,8017,22.5,15,1,6,350,0,...,22.115048,6.243104,10.195583,3.265523,4.448919,4.378773,2.345736,1.257296,-3.991786,-3.043323
4,0.72,6.733333,68,2900,22.5,15,3,6,300,0,...,26.607399,7.497952,12.391656,5.850002,4.529510,6.236501,4.371455,1.426839,-2.397728,-4.159834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.88,8.650000,74,6536,21.0,20,1,6,400,4,...,55.781423,-34.890406,24.910494,-24.711970,31.871810,-24.889958,22.753106,-11.470411,7.938448,-7.446821
496,0.89,8.500000,74,5628,21.0,20,1,5,300,4,...,47.145010,-41.015930,19.350729,-28.172226,30.483321,-25.031947,27.416434,-7.228113,12.078870,-8.351442
497,0.75,8.266667,74,595,21.0,20,1,5,300,2,...,53.863180,-32.002747,27.553545,-24.016376,35.167236,-25.989636,23.615290,-13.277489,11.329945,-10.064979
498,0.83,7.833333,74,7499,21.0,20,1,4,200,2,...,36.533682,-44.521168,20.733773,-28.723805,31.096191,-21.767701,26.089461,-10.327917,11.956299,-10.462354


In [62]:
columns_to_keep = ['Sleep Efficiency (%)', 'snoring_mfcc_0', 'snoring_mfcc_1', 'snoring_mfcc_2', 'snoring_mfcc_3',
                   'snoring_mfcc_4', 'snoring_mfcc_5', 'snoring_mfcc_6', 'snoring_mfcc_7', 'snoring_mfcc_8',
                   'snoring_mfcc_9', 'snoring_mfcc_10', 'snoring_mfcc_11', 'snoring_mfcc_12', 'non_snoring_mfcc_0', 'non_snoring_mfcc_1', 'non_snoring_mfcc_2', 'non_snoring_mfcc_3',
                   'non_snoring_mfcc_4', 'non_snoring_mfcc_5', 'non_snoring_mfcc_6', 'non_snoring_mfcc_7',
                   'non_snoring_mfcc_8', 'non_snoring_mfcc_9', 'non_snoring_mfcc_10', 'non_snoring_mfcc_11',
                   'non_snoring_mfcc_12', 'Insomnia (Yes/No)_NO', 'Insomnia (Yes/No)_YES']

In [63]:
df_one_hot_subset = df_one_hot[columns_to_keep]

In [64]:
df_one_hot_subset

,Sleep Efficiency (%),snoring_mfcc_0,snoring_mfcc_1,snoring_mfcc_2,snoring_mfcc_3,snoring_mfcc_4,snoring_mfcc_5,snoring_mfcc_6,snoring_mfcc_7,snoring_mfcc_8,...,non_snoring_mfcc_5,non_snoring_mfcc_6,non_snoring_mfcc_7,non_snoring_mfcc_8,non_snoring_mfcc_9,non_snoring_mfcc_10,non_snoring_mfcc_11,non_snoring_mfcc_12,Insomnia (Yes/No)_NO,Insomnia (Yes/No)_YES
0,1.00,451.310674,-12.855848,-78.177468,42.645191,-38.153833,7.666822,17.426707,0.525363,-2.136169,...,20.378508,-11.392241,20.031550,-2.818156,-2.804502,6.367196,-2.627417,0.926880,1,0
1,0.03,312.302241,3.723321,-59.274932,15.770150,-32.144009,3.124675,15.789284,4.992782,-1.527913,...,23.600391,-23.501267,6.519634,5.660507,-2.793761,12.978388,-9.589830,-7.780554,0,1
2,0.98,428.503926,-4.580862,-45.462593,51.781359,-28.381126,10.663609,16.927527,7.173176,1.961152,...,21.859583,-36.483370,19.489034,1.779462,-4.483938,-4.372021,5.027823,-6.741121,1,0
3,0.65,294.105113,24.836825,-40.771665,37.425360,-6.546441,21.410731,11.138267,-6.847575,6.156239,...,10.195583,3.265523,4.448919,4.378773,2.345736,1.257296,-3.991786,-3.043323,0,1
4,0.72,454.553301,67.419677,-84.586400,21.192197,-13.967239,16.276450,10.347922,-9.177961,0.467334,...,12.391656,5.850002,4.529510,6.236501,4.371455,1.426839,-2.397728,-4.159834,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.88,444.447665,68.092312,-75.481824,33.794516,-4.984247,20.269710,14.677384,-9.182213,0.280679,...,24.910494,-24.711970,31.871810,-24.889958,22.753106,-11.470411,7.938448,-7.446821,1,0
496,0.89,451.586562,65.943322,-83.101528,21.566735,-10.011301,11.613707,10.176297,-8.044793,-3.343425,...,19.350729,-28.172226,30.483321,-25.031947,27.416434,-7.228113,12.078870,-8.351442,1,0
497,0.75,223.645045,10.566871,-44.319803,24.748280,-12.833769,15.387709,7.385326,-4.233281,11.804925,...,27.553545,-24.016376,35.167236,-25.989636,23.615290,-13.277489,11.329945,-10.064979,0,1
498,0.83,452.876671,69.845296,-80.012503,25.591431,-8.008599,16.423274,13.290543,-8.042622,2.937176,...,20.733773,-28.723805,31.096191,-21.767701,26.089461,-10.327917,11.956299,-10.462354,1,0


In [65]:
y = df_one_hot[['Insomnia (Yes/No)_NO', 'Insomnia (Yes/No)_YES']]

In [66]:
y

,Insomnia (Yes/No)_NO,Insomnia (Yes/No)_YES
0,1,0
1,0,1
2,1,0
3,0,1
4,0,1
...,...,...
495,1,0
496,1,0
497,0,1
498,1,0


In [67]:
x = df_one_hot_subset.drop(['Insomnia (Yes/No)_NO', 'Insomnia (Yes/No)_YES'], axis=1)

In [68]:
x

,Sleep Efficiency (%),snoring_mfcc_0,snoring_mfcc_1,snoring_mfcc_2,snoring_mfcc_3,snoring_mfcc_4,snoring_mfcc_5,snoring_mfcc_6,snoring_mfcc_7,snoring_mfcc_8,...,non_snoring_mfcc_3,non_snoring_mfcc_4,non_snoring_mfcc_5,non_snoring_mfcc_6,non_snoring_mfcc_7,non_snoring_mfcc_8,non_snoring_mfcc_9,non_snoring_mfcc_10,non_snoring_mfcc_11,non_snoring_mfcc_12
0,1.00,451.310674,-12.855848,-78.177468,42.645191,-38.153833,7.666822,17.426707,0.525363,-2.136169,...,-9.269381,-18.835488,20.378508,-11.392241,20.031550,-2.818156,-2.804502,6.367196,-2.627417,0.926880
1,0.03,312.302241,3.723321,-59.274932,15.770150,-32.144009,3.124675,15.789284,4.992782,-1.527913,...,13.897652,-1.592268,23.600391,-23.501267,6.519634,5.660507,-2.793761,12.978388,-9.589830,-7.780554
2,0.98,428.503926,-4.580862,-45.462593,51.781359,-28.381126,10.663609,16.927527,7.173176,1.961152,...,-4.694515,-2.959561,21.859583,-36.483370,19.489034,1.779462,-4.483938,-4.372021,5.027823,-6.741121
3,0.65,294.105113,24.836825,-40.771665,37.425360,-6.546441,21.410731,11.138267,-6.847575,6.156239,...,22.115048,6.243104,10.195583,3.265523,4.448919,4.378773,2.345736,1.257296,-3.991786,-3.043323
4,0.72,454.553301,67.419677,-84.586400,21.192197,-13.967239,16.276450,10.347922,-9.177961,0.467334,...,26.607399,7.497952,12.391656,5.850002,4.529510,6.236501,4.371455,1.426839,-2.397728,-4.159834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.88,444.447665,68.092312,-75.481824,33.794516,-4.984247,20.269710,14.677384,-9.182213,0.280679,...,55.781423,-34.890406,24.910494,-24.711970,31.871810,-24.889958,22.753106,-11.470411,7.938448,-7.446821
496,0.89,451.586562,65.943322,-83.101528,21.566735,-10.011301,11.613707,10.176297,-8.044793,-3.343425,...,47.145010,-41.015930,19.350729,-28.172226,30.483321,-25.031947,27.416434,-7.228113,12.078870,-8.351442
497,0.75,223.645045,10.566871,-44.319803,24.748280,-12.833769,15.387709,7.385326,-4.233281,11.804925,...,53.863180,-32.002747,27.553545,-24.016376,35.167236,-25.989636,23.615290,-13.277489,11.329945,-10.064979
498,0.83,452.876671,69.845296,-80.012503,25.591431,-8.008599,16.423274,13.290543,-8.042622,2.937176,...,36.533682,-44.521168,20.733773,-28.723805,31.096191,-21.767701,26.089461,-10.327917,11.956299,-10.462354


In [69]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

In [70]:
x_train

,Sleep Efficiency (%),snoring_mfcc_0,snoring_mfcc_1,snoring_mfcc_2,snoring_mfcc_3,snoring_mfcc_4,snoring_mfcc_5,snoring_mfcc_6,snoring_mfcc_7,snoring_mfcc_8,...,non_snoring_mfcc_3,non_snoring_mfcc_4,non_snoring_mfcc_5,non_snoring_mfcc_6,non_snoring_mfcc_7,non_snoring_mfcc_8,non_snoring_mfcc_9,non_snoring_mfcc_10,non_snoring_mfcc_11,non_snoring_mfcc_12
205,0.77,485.969722,-44.534062,-92.347563,21.651008,-25.976546,17.404928,-24.706326,9.907720,18.366620,...,31.233657,-23.539039,40.176216,-27.313141,25.457805,-18.586578,15.948539,-12.430309,8.925362,-8.035423
404,0.74,562.594251,-34.342987,-91.134522,18.299387,-28.048868,18.093229,-15.109179,40.948568,6.354852,...,-23.925827,4.059194,2.383029,25.356525,17.133428,-1.772723,0.103863,-6.666801,-2.123785,-5.779611
337,0.79,548.613235,-26.917069,-42.642645,45.392212,2.315683,-5.223191,-4.356089,24.616563,-8.292950,...,48.243930,-15.835414,37.194381,-20.655890,10.190953,-19.524356,6.479382,-11.467064,6.529185,-10.622795
440,0.60,558.495606,-55.633790,-41.868906,49.995881,-47.907279,38.148373,-15.197273,34.528014,-12.715008,...,103.975674,-41.537882,5.554141,12.728680,-11.147058,13.233788,-24.948244,0.126973,-13.469790,-11.864336
55,0.99,565.065772,57.207890,-46.402025,17.768915,-20.978460,15.465333,-12.208533,1.967428,-19.035123,...,-13.731749,-32.844017,14.477455,-5.771065,3.369322,-9.585159,-0.321407,0.836981,-3.468166,-4.912932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,0.83,615.671591,-16.114873,-47.032703,41.645114,-1.566814,-6.073278,-4.070736,24.534614,-8.106686,...,48.224194,-21.367036,37.824181,-23.304110,20.896882,-9.275458,15.120160,-5.862084,6.721080,-3.291375
359,1.00,495.397608,4.288272,-58.144189,61.316954,-47.197012,17.541800,-16.393692,10.905431,21.650417,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
323,0.75,461.954401,24.029046,-84.337772,20.879277,-26.289241,18.986912,-20.639102,2.946626,6.348151,...,-14.383825,-24.749906,2.148386,-11.171422,18.060693,13.917017,-2.551473,-5.407091,-1.552163,-5.797444
280,0.92,624.396951,65.372601,-34.195709,14.409525,-29.875742,15.874826,-9.851792,-2.404550,-24.405974,...,6.468638,-12.236467,7.979056,-6.244521,4.978175,-12.044107,-5.310660,-4.407416,1.473952,-9.459680


In [71]:
x_test

,Sleep Efficiency (%),snoring_mfcc_0,snoring_mfcc_1,snoring_mfcc_2,snoring_mfcc_3,snoring_mfcc_4,snoring_mfcc_5,snoring_mfcc_6,snoring_mfcc_7,snoring_mfcc_8,...,non_snoring_mfcc_3,non_snoring_mfcc_4,non_snoring_mfcc_5,non_snoring_mfcc_6,non_snoring_mfcc_7,non_snoring_mfcc_8,non_snoring_mfcc_9,non_snoring_mfcc_10,non_snoring_mfcc_11,non_snoring_mfcc_12
69,0.95,642.186210,85.631195,-50.379516,23.530583,-27.140805,23.196974,-6.974516,1.481833,-26.988794,...,18.634912,-35.502301,6.134250,-12.559722,-3.728098,-11.335638,-2.691290,-6.818483,-5.267171,-5.522529
29,0.67,517.062618,-47.928577,-43.063272,47.015549,-34.783658,30.802037,-11.747539,29.338113,-12.837725,...,2.922385,-15.666132,16.525341,4.769846,11.064410,1.518571,12.426700,3.118235,9.679657,-1.627687
471,0.69,187.486647,8.520929,-33.241834,34.067603,0.215394,25.586067,11.653099,-2.712664,2.557731,...,39.915774,-35.025388,29.743602,-24.287907,34.816597,-22.731721,20.038354,-9.030225,12.477653,-4.346829
344,0.74,554.387003,-15.764366,-48.872981,33.022467,-3.745431,-5.832793,-4.566380,18.029208,-5.171748,...,36.131832,-32.485982,35.944132,-20.920512,15.185328,-11.695358,10.541969,-4.137138,9.299827,-2.150827
54,0.69,620.565739,68.859342,-42.738658,19.958127,-18.391224,21.083279,-11.501635,-3.528656,-22.491923,...,-8.964693,-40.414419,6.917034,-8.179518,4.227654,-7.817339,-3.333739,5.767799,0.054462,2.000405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,0.61,415.435045,16.228600,-72.359351,24.531731,-16.055263,24.386976,-11.740403,2.729154,9.749271,...,44.133806,-30.783790,35.039482,-22.187511,33.416730,-19.637835,20.645553,-11.767382,7.143608,-7.900916
152,0.90,651.414049,-39.342176,-18.139295,67.092585,-16.932653,52.276570,-29.296329,19.164436,-3.968731,...,-36.280571,-2.260406,5.186033,18.496262,15.084716,-4.744411,-17.964847,-10.828371,3.460424,16.034257
154,0.67,568.707905,-17.581511,-9.029940,69.968342,-35.487631,37.725911,-21.365216,28.710748,-2.023235,...,-20.645690,-15.602205,-28.245383,-11.020392,6.511380,25.413027,21.205209,2.932340,-11.346265,-14.079382
56,0.82,651.759353,68.932632,-41.588206,29.602827,-18.466437,18.591472,-5.367318,1.088039,-17.967327,...,8.295886,-52.309721,24.368815,-18.365801,11.988186,-2.759347,-1.188487,3.986544,-2.679438,-6.393463


In [72]:
clf = RandomForestClassifier(random_state=100)

In [73]:
clf = clf.fit(x_train, y_train)

In [74]:
predictions = clf.predict(x_test)
predictions

array([[1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0,

In [75]:
accuracy_score(y_test, predictions)

1.0

In [87]:
report = classification_report(y_test, predictions, target_names=["Insomnia (Yes/No)_NO", "Insomnia (Yes/No)_YES,"])

In [88]:
report

'                        precision    recall  f1-score   support\n\n  Insomnia (Yes/No)_NO       1.00      1.00      1.00        36\nInsomnia (Yes/No)_YES,       1.00      1.00      1.00        64\n\n             micro avg       1.00      1.00      1.00       100\n             macro avg       1.00      1.00      1.00       100\n          weighted avg       1.00      1.00      1.00       100\n           samples avg       1.00      1.00      1.00       100\n'

In [86]:
with open('classification_report.txt', 'w') as f:
    f.write(report)

In [78]:
scores = cross_val_score(clf, x, y, cv=5)

In [79]:
scores

array([1., 1., 1., 1., 1.])

In [80]:
joblib.dump(clf, 'sleep_analysis_model1.joblib')

['sleep_analysis_model1.joblib']

In [ ]:
model = joblib.load('sleep_analysis_model1.joblib')